In [3]:
import datetime as dt

import pandas as pd
import pandas_market_calendars as mcal
import polars as pl

nse = mcal.get_calendar("NSE")

df = pd.read_csv("Check Rolling Pivots TB v1.csv")

In [4]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = '{:.4f}'.format

In [5]:
df.head(30)

,Unnamed: 0.1,Unnamed: 0,Trade Type,Signal Generated At,Entry Time,Entry Price,Initial SL,Exit Time,Exit Price,Remarks,Points Captured,Points w CS,Qty,Leverage,PnL,PnL w CS,ROI%,ROI% w CS,Trade Year,Index,DD%,Strike,Option Type,Expiry,Option Contract,Instrument,Option Entry Price,Option Exit Price,Option Points,Options PnL,Option Points w CS,Options PnL w CS,Options ROI%,Options ROI% w CS
0,0,1789,SHORT,2018-01-01 11:15:00+05:30,2018-01-01 12:15:00+05:30,10521.1500,10537.8500,2018-01-05 09:15:00+05:30,10537.8500,Initial SL Hit,-16.7000,-20.9118,380.1866,4,-6349.1158,-7950.3857,-0.6349,-0.3975,2018,NIFTY,0.0000,10500,CE,2018-01-11,NIFTY1811110500CE,nifty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,SHORT,2018-01-01 15:15:00+05:30,2018-01-02 09:15:00+05:30,25271.5500,25557.6500,2018-01-04 12:15:00+05:30,25434.3000,Reversed,-162.7500,-167.8206,118.7106,3,-19320.1446,-19922.0766,-1.9320,-0.9961,2018,BANKNIFTY,-0.9961,25300,CE,2018-01-04,BANKNIFTY1810425300CE,bnf,160.0000,113.2500,46.7500,5549.7190,44.0175,5225.3423,0.5550,0.5225
2,2,1,LONG,2018-01-04 11:15:00+05:30,2018-01-04 12:15:00+05:30,25434.3000,25310.3000,2018-01-05 11:15:00+05:30,25529.5000,Reversed,95.2000,90.1036,117.9510,3,11228.9310,10627.8081,1.1229,0.5314,2018,BANKNIFTY,-0.4647,25400,PE,2018-01-11,BANKNIFTY1811125400PE,bnf,137.0000,77.7000,59.3000,6994.4917,57.1530,6741.2510,0.6994,0.6741
3,3,2,SHORT,2018-01-05 10:15:00+05:30,2018-01-05 11:15:00+05:30,25529.5000,25575.2500,2018-01-05 13:15:00+05:30,25570.8000,Reversed,-41.3000,-46.4100,117.5111,3,-4853.2090,-5453.6944,-0.4853,-0.2727,2018,BANKNIFTY,-0.7374,25500,CE,2018-01-11,BANKNIFTY1811125500CE,bnf,148.0000,146.0000,2.0000,235.0222,-0.9400,-110.4604,0.0235,-0.0110
4,4,1790,SHORT,2018-01-05 10:15:00+05:30,2018-01-05 11:15:00+05:30,10544.9000,10562.8000,2018-01-05 13:15:00+05:30,10540.6500,Reversed,4.2500,0.0329,379.3303,4,1612.1537,12.4762,0.1612,0.0006,2018,NIFTY,-0.7368,10550,CE,2018-01-11,NIFTY1811110550CE,nifty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,3,LONG,2018-01-05 12:15:00+05:30,2018-01-05 13:15:00+05:30,25570.8000,25508.4000,2018-01-08 11:15:00+05:30,25717.8500,Reversed,147.0500,141.9211,117.3213,3,17252.1001,16650.3748,1.7252,0.8325,2018,BANKNIFTY,0.0000,25600,PE,2018-01-11,BANKNIFTY1811125600PE,bnf,162.3000,66.0000,96.3000,11298.0431,94.0170,11030.1985,1.1298,1.1030
6,6,1791,LONG,2018-01-05 12:15:00+05:30,2018-01-05 13:15:00+05:30,10540.6500,10520.1000,2018-01-08 13:15:00+05:30,10609.7500,Reversed,69.1000,64.8699,379.4832,4,26222.2918,24617.0473,2.6222,1.2309,2018,NIFTY,0.0000,10550,PE,2018-01-11,NIFTY1811110550PE,nifty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,4,SHORT,2018-01-08 10:15:00+05:30,2018-01-08 11:15:00+05:30,25717.8500,25771.5000,2018-01-09 09:15:00+05:30,25771.5000,Initial SL Hit,-53.6500,-58.7989,116.6505,3,-6258.2992,-6858.9250,-0.6258,-0.3429,2018,BANKNIFTY,-0.3429,25700,CE,2018-01-11,BANKNIFTY1811125700CE,bnf,126.0000,97.0500,28.9500,3377.0319,26.7195,3116.8430,0.3377,0.3117
8,8,1792,SHORT,2018-01-08 12:15:00+05:30,2018-01-08 13:15:00+05:30,10609.7500,10623.2000,2018-01-08 14:15:00+05:30,10623.2000,Initial SL Hit,-13.4500,-17.6966,377.0117,4,-5070.8075,-6671.8217,-0.5071,-0.3336,2018,NIFTY,-0.6765,10600,CE,2018-01-11,NIFTY1811110600CE,nifty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,1793,SHORT,2018-01-08 15:15:00+05:30,2018-01-09 09:15:00+05:30,10614.5500,10631.2000,2018-01-09 09:15:00+05:30,10645.8000,Gap Outside Initial SL,-31.2500,-35.5021,376.8412,4,-11776.2882,-13378.6435,-1.1776,-0.6689,2018,NIFTY,-1.3455,10600,CE,2018-01-11,NIFTY1811110600CE,nifty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
bnf_1min = pd.read_csv('../data/bnf_min.csv')
bnf_1min.head(30)

,datetime,open,high,low,close,volume
0,2017-01-02T09:15:00.000000,18242.3000,18248.2000,18175.9000,18181.2000,0
1,2017-01-02T09:16:00.000000,18181.8500,18194.7000,18179.9500,18184.4500,0
2,2017-01-02T09:17:00.000000,18184.9500,18189.2500,18133.8000,18133.8000,0
3,2017-01-02T09:18:00.000000,18135.1000,18141.5500,18118.5500,18138.9500,0
4,2017-01-02T09:19:00.000000,18138.9500,18142.5500,18120.4500,18124.3000,0
5,2017-01-02T09:20:00.000000,18123.9500,18123.9500,18100.3000,18100.3000,0
6,2017-01-02T09:21:00.000000,18101.6500,18112.1500,18091.4500,18112.1000,0
7,2017-01-02T09:22:00.000000,18114.3500,18116.2000,18091.6500,18101.6000,0
8,2017-01-02T09:23:00.000000,18103.6500,18115.1000,18094.8000,18102.9000,0
9,2017-01-02T09:24:00.000000,18102.4500,18102.9000,18089.3500,18093.0500,0


In [22]:
import pandas as pd

# Load tradebook DataFrame
tradebook = df
tradebook['Entry Time'] = pd.to_datetime(tradebook['Entry Time'])
tradebook['Exit Time'] = pd.to_datetime(tradebook['Exit Time'])
# Ensure 'Timestamp' column is in datetime format
bnf_1min['datetime'] = pd.to_datetime(bnf_1min['datetime']).dt.tz_localize(None)

# Function to find breach time
def find_entry_breach_time(df, price, trade_type):
    print(price)
    if trade_type == "LONG":
        # breach_time = df[df['close'] > price].iloc[0]['Timestamp'] if not df[df['close'] > price].empty else None
        breach_time = df[df['high'] >= price].iloc[0]['datetime']
    else:
        breach_time = df[df['low'] <= price].iloc[0]['datetime']
    return breach_time

def find_exit_breach_time(df, price, trade_type):
    print(price)
    if trade_type == "LONG":
        # breach_time = df[df['close'] > price].iloc[0]['Timestamp'] if not df[df['close'] > price].empty else None
        breach_time = df[df['low'] <= price].iloc[0]['datetime']
    else:
        breach_time = df[df['high'] >= price].iloc[0]['datetime']
    return breach_time

# Iterate over each trade
for index, row in tradebook.iterrows():
    if row['Index'] == "BANKNIFTY":
        print(row)
        entry_time = row['Entry Time'].tz_localize(None)
        exit_time = row['Exit Time'].tz_localize(None)
        
        # Filter data for entry timeframe (from entry_time to entry_time + 1 hour)
        entry_start = entry_time
        entry_end = entry_time + dt.timedelta(hours=1)
        
        entry_data = bnf_1min[(bnf_1min['datetime'] >= entry_start) & (bnf_1min['datetime'] <= entry_end)]
        print(entry_data)
        # Find Entry Breach Time
        entry_breach_time = find_entry_breach_time(entry_data, row['Entry Price'], row['Trade Type'])
        
        # Filter data for exit timeframe (from exit_time to exit_time + 1 hour)
        # exit_start = exit_time
        # exit_end = exit_time + dt.timedelta(hours=1)
        
        # exit_data = bnf_1min[(bnf_1min['datetime'] >= exit_start) & (bnf_1min['datetime'] <= exit_end)]
        # print(exit_data)
        # # Find Exit Breach Time
        # exit_breach_time = find_exit_breach_time(exit_data, row['Exit Price'], row['Trade Type'])
        
        # Update the tradebook with new timestamps
        tradebook.at[index, 'Entry Minute Timestamp'] = entry_breach_time
        # tradebook.at[index, 'Exit Minute Timestamp'] = exit_breach_time

# Save updated tradebook DataFrame
tradebook.to_csv('updated_tradebook.csv', index=False)

Unnamed: 0.1                                      1
Unnamed: 0                                        0
Trade Type                                    SHORT
Signal Generated At       2018-01-01 15:15:00+05:30
Entry Time                2018-01-02 09:15:00+05:30
Entry Price                              25271.5500
Initial SL                               25557.6500
Exit Time                 2018-01-04 12:15:00+05:30
Exit Price                               25434.3000
Remarks                                    Reversed
Points Captured                           -162.7500
Points w CS                               -167.8206
Qty                                        118.7106
Leverage                                          3
PnL                                     -19320.1446
PnL w CS                                -19922.0766
ROI%                                        -1.9320
ROI% w CS                                   -0.9961
Trade Year                                     2018
Index       

IndexError: single positional indexer is out-of-bounds